[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/dgunning/edgartools/blob/main/notebooks/download-sec-filings-bulk-python.ipynb)

# Download SEC Filings in Bulk with Python -- Free, No API Key

Use **edgartools** to download SEC filings in bulk -- completely free, no API key or paid subscription required. Search by company, form type, or date range, then save filings as text, markdown, HTML, or ZIP archives.

**What you'll learn:**
- Download any company's filings by form type
- Search filings by date range, year, or quarter
- Save filing content as text, markdown, or HTML
- Download all filing attachments to a folder or ZIP
- Save and reload filings for offline analysis
- Build a dataset of filings across multiple companies

## Install edgartools

In [1]:
!pip install -U edgartools


[notice] A new release of pip is available: 24.0 -> 26.0.1
[notice] To update, run: pip install --upgrade pip


## Setup

The SEC requires all automated tools to identify themselves. Replace the email below with your own -- any valid email works.

In [2]:
from pathlib import Path
from edgar import *

# The SEC requires you to identify yourself (any email works)
set_identity("your.name@example.com")

## Download a Filing in 3 Lines

Get any company's latest filing and extract its text -- ready for analysis, NLP, or storage:

In [3]:
filing = Company("MSFT").get_filings(form="10-K")[0]
text = filing.text()

print(f"Company: {filing.company}")
print(f"Form:    {filing.form}")
print(f"Date:    {filing.filing_date}")
print(f"Length:  {len(text):,} characters")
print()
print(text[:500])

Company: MICROSOFT CORP
Form:    10-K
Date:    2025-07-30
Length:  349,397 characters

UNITED STATES

SECURITIES AND EXCHANGE COMMISSION

Washington, D. C. 20549

FORM10-K

  ANNUAL REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934  
 ────────────────────────────────────────────────────────────────────────────────────────

  For the Fiscal Year Ended June 30, 2025                                                   
  TRANSITION REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934  
 ───────────────────────────────────────────────


## Choose Your Format: Text, Markdown, or HTML

Every filing can be extracted in three formats depending on your use case:

In [4]:
# Plain text -- best for NLP, search, and text analysis
text = filing.text()
print(f"Text:     {len(text):>10,} chars")

# Markdown -- best for readable documents and LLM context
markdown = filing.markdown()
print(f"Markdown: {len(markdown):>10,} chars")

# HTML -- best for preserving original formatting and tables
html = filing.html()
print(f"HTML:     {len(html):>10,} chars")

Text:        349,397 chars
Markdown:    421,988 chars
HTML:      8,158,067 chars


## Save Filing Content to Files

Write filing content to disk in any format:

In [5]:
import os

# Create output directory
os.makedirs("filings_output", exist_ok=True)

# Save as plain text
with open("filings_output/msft_10k.txt", "w") as f:
    f.write(filing.text())

# Save as markdown
with open("filings_output/msft_10k.md", "w") as f:
    f.write(filing.markdown())

# Check file sizes
for name in ["msft_10k.txt", "msft_10k.md"]:
    size = os.path.getsize(f"filings_output/{name}")
    print(f"  {name:20s} {size:>10,} bytes")

  msft_10k.txt            363,075 bytes
  msft_10k.md             422,974 bytes


## Download All Filing Attachments

Each SEC filing contains multiple documents -- the primary filing, exhibits, XBRL data, and more. Download everything at once:

In [6]:
# See what's in the filing
filing.attachments

                                                       Attachments                                                 
                                                                                                                   
  Seq   Document                        Description                                                    Type        
 ──────────────────────────────────────────────────────────────────────────────────────────────────────────────────
  1     msft-20250630.htm               10-K                                                           📜  10-K    
  2     msft-ex10_7.htm                 EX-10.7                                                        📋  EX-10.7 
  3     msft-ex10_8.htm                 EX-10.8                                                        📋  EX-10.8 
  4     msft-ex21.htm                   EX-21                                                          📋  EX-21   
  5     msft-ex23_1.htm                 EX-23.1                             

In [7]:
# Download all attachments to a folder
output_dir = Path("filings_output/msft_10k_attachments")
output_dir.mkdir(parents=True, exist_ok=True)

filing.attachments.download(output_dir)

files = sorted(output_dir.iterdir())
print(f"Downloaded {len(files)} files:")
for f in files[:10]:
    print(f"  {f.name:50s} {f.stat().st_size:>10,} bytes")
if len(files) > 10:
    print(f"  ... and {len(files) - 10} more")

Downloaded 126 files:
  0000950170-25-100235-xbrl.zip                         547,983 bytes
  FilingSummary.xml                                      67,404 bytes
  MetaLinks.json                                      1,445,698 bytes
  R1.htm                                                 83,876 bytes
  R10.htm                                                61,279 bytes
  R100.htm                                               37,779 bytes
  R101.htm                                               12,702 bytes
  R102.htm                                               27,657 bytes
  R103.htm                                               14,335 bytes
  R104.htm                                                7,135 bytes
  ... and 116 more


In [8]:
# Or download as a ZIP archive
zip_path = Path("filings_output/msft_10k.zip")
filing.attachments.download(zip_path, archive=True)

print(f"ZIP archive: {zip_path.stat().st_size:,} bytes")

ZIP archive: 34,679,090 bytes


## Save and Reload Filings for Offline Use

Use `.save()` to serialize a filing as a self-contained pickle file. Load it later with `Filing.load()` -- no network access required:

In [9]:
# Save filing for offline use
filing.save("filings_output")

# Later, load without any network access
loaded = Filing.load(f"filings_output/{filing.accession_no}.pkl")

print(f"Loaded: {loaded.company} {loaded.form} {loaded.filing_date}")
print(f"Text available offline: {len(loaded.text()):,} chars")

Loaded: MICROSOFT CORP 10-K 2025-07-30
Text available offline: 349,397 chars


## Search Filings by Date Range

Use `get_filings()` with date ranges to find filings across all companies:

In [10]:
# All 10-K filings from a date range
filings = get_filings(filing_date="2025-12-01:2025-12-05", form="10-K")
print(f"10-K filings Dec 1-5, 2025: {len(filings)}")

for f in filings.head(10):
    print(f"  {f.company:45s} {f.form:6s} {f.filing_date}")

10-K filings Dec 1-5, 2025: 20
  COOPER COMPANIES, INC.                        10-K   2025-12-05
  HORMEL FOODS CORP /DE/                        10-K   2025-12-05
  Inotiv, Inc.                                  10-K   2025-12-05
  JANEL CORP                                    10-K   2025-12-05
  SRx Health Solutions, Inc.                    10-K   2025-12-05
  Armada Acquisition Corp. II                   10-K   2025-12-04
  Azenta, Inc.                                  10-K   2025-12-04
  GOLUB CAPITAL DIRECT LENDING CORP             10-K   2025-12-04
  RGC RESOURCES INC                             10-K   2025-12-04
  Crown PropTech Acquisitions                   10-K   2025-12-03


In [11]:
# All 10-K filings from a specific quarter
q4_filings = get_filings(year=2025, quarter=4, form="10-K")
print(f"10-K filings in Q4 2025: {len(q4_filings)}")
q4_filings.head(5)

10-K filings in Q4 2025: 409


╭────────────────────────────────────────────────── SEC Filings ──────────────────────────────────────────────────╮
│                                                                                                                 │
│                                                                            Filing                               │
│    Form              CIK   Tick…   Company                                 Date         Accession Number        │
│  ─────────────────────────────────────────────────────────────────────────────────────────────────────────────  │
│    10-K          1502557   KPEA    Kun Peng International Ltd.             2025-12-31   0001493152-25-029…      │
│    10-K            72633   NRT     NORTH EUROPEAN OIL ROYALTY TRUST        2025-12-31   0001140361-25-047…      │
│    10-K          2016167           Transit Pro Tech Inc.                   2025-12-31   0001753926-25-002…      │
│    10-K          2038439   VWAV    VisionWave Holdings, Inc.          

## Batch Download: Multiple Companies

Loop through a list of companies and save their latest filings:

In [12]:
import os

tickers = ["AAPL", "MSFT", "GOOG", "AMZN", "NVDA"]
os.makedirs("filings_output/10k_texts", exist_ok=True)

for ticker in tickers:
    filing = Company(ticker).get_filings(form="10-K")[0]
    text = filing.text()

    filename = f"filings_output/10k_texts/{ticker}_10k.txt"
    with open(filename, "w") as f:
        f.write(text)

    print(f"  {ticker:6s} {filing.filing_date}  {len(text):>10,} chars  -> {filename}")

print(f"\nSaved {len(tickers)} filings")

  AAPL   2025-10-31     261,019 chars  -> filings_output/10k_texts/AAPL_10k.txt
  MSFT   2025-07-30     349,397 chars  -> filings_output/10k_texts/MSFT_10k.txt


  GOOG   2026-02-05     447,697 chars  -> filings_output/10k_texts/GOOG_10k.txt


  AMZN   2026-02-06     330,396 chars  -> filings_output/10k_texts/AMZN_10k.txt


  NVDA   2025-02-26     423,623 chars  -> filings_output/10k_texts/NVDA_10k.txt

Saved 5 filings


## Download Specific Exhibits

Many filings include important exhibits -- subsidiary lists (EX-21), certifications (EX-31/32), or press releases (EX-99). Browse and download individual exhibits:

In [13]:
# List all exhibits in Microsoft's 10-K
filing = Company("MSFT").get_filings(form="10-K")[0]
exhibits = filing.attachments.exhibits

print(f"Exhibits in {filing.company} {filing.form}:")
for ex in exhibits:
    print(f"  {ex.document_type:15s} {ex.document}")

# Download the subsidiary list (EX-21)
for ex in exhibits:
    if ex.document_type == "EX-21":
        content = ex.download()
        print(f"\nEX-21 (subsidiaries): {len(content):,} chars")
        break

Exhibits in MICROSOFT CORP 10-K:
  10-K            msft-20250630.htm
  EX-10.7         msft-ex10_7.htm
  EX-10.8         msft-ex10_8.htm
  EX-21           msft-ex21.htm
  EX-23.1         msft-ex23_1.htm
  EX-31.1         msft-ex31_1.htm
  EX-31.2         msft-ex31_2.htm
  EX-32.1         msft-ex32_1.htm
  EX-32.2         msft-ex32_2.htm

EX-21 (subsidiaries): 15,795 chars


## Clean Up

Remove the output files created during this tutorial:

In [14]:
import shutil
shutil.rmtree("filings_output", ignore_errors=True)
print("Cleaned up filings_output/")

Cleaned up filings_output/


## Why EdgarTools?

EdgarTools is free and open-source. Compare downloading SEC filings:

**With edgartools (free, no API key):**
```python
filing = Company("AAPL").get_filings(form="10-K")[0]
filing.text()                          # Plain text
filing.markdown()                      # Markdown
filing.attachments.download(output_dir) # All documents
```

**Typical paid API approach ($50+/month, API key required):**
```python
import requests
from sec_api import QueryApi, RenderApi
query_api = QueryApi(api_key="YOUR_PAID_API_KEY")
render_api = RenderApi(api_key="YOUR_PAID_API_KEY")
# ... build EDGAR query syntax, paginate, extract URLs, download individually
```

With edgartools, downloading filings is simple Python -- search, filter, and save in any format, no query syntax, no API key, no monthly fee.

## Quick Reference

```python
from pathlib import Path
from edgar import *
set_identity("your.name@example.com")

# ── Get filings ──
filing = Company("AAPL").get_filings(form="10-K")[0]
filings = get_filings(filing_date="2025-12-01:2025-12-05", form="10-K")
filings = get_filings(year=2025, quarter=4, form="10-K")

# ── Extract content ──
filing.text()                           # Plain text
filing.markdown()                       # Markdown
filing.html()                           # Raw HTML

# ── Download documents ──
filing.attachments.download(Path("./output"))           # All to folder
filing.attachments.download(Path("filing.zip"), archive=True)  # As ZIP

# ── Download individual exhibits ──
for ex in filing.attachments.exhibits:  # Browse exhibits
    content = ex.download()              # Download one

# ── Save for offline use ──
filing.save("./output")                # Serialize to pickle
filing = Filing.load("output/accession.pkl")  # Load offline
```

## What's Next

You've learned how to download SEC filings in bulk with Python. Here are related tutorials:

- [Download and Parse 10-K Annual Reports](https://colab.research.google.com/github/dgunning/edgartools/blob/main/notebooks/download-10k-annual-report-python.ipynb)
- [Get Today's SEC Filings with Python](https://colab.research.google.com/github/dgunning/edgartools/blob/main/notebooks/sec-filings-today-python.ipynb)
- [Search SEC Filings by Ticker Symbol](https://colab.research.google.com/github/dgunning/edgartools/blob/main/notebooks/Ticker-Search-with-edgartools.ipynb)
- [Extract Revenue and Earnings from SEC Filings](https://colab.research.google.com/github/dgunning/edgartools/blob/main/notebooks/extract-revenue-earnings-python.ipynb)

**Resources:**
- [EdgarTools Documentation](https://edgartools.readthedocs.io/)
- [GitHub Repository](https://github.com/dgunning/edgartools)
- [PyPI Package](https://pypi.org/project/edgartools/)

---

## Support EdgarTools

If you found this tutorial helpful, here are a few ways to support the project:

- **Star the repo** -- [github.com/dgunning/edgartools](https://github.com/dgunning/edgartools) -- it helps others discover edgartools
- **Visit edgartools.io** -- [edgartools.io](https://www.edgartools.io/) -- for more tutorials, articles, and updates
- **Report issues** -- found a bug or have a feature idea? [Open an issue](https://github.com/dgunning/edgartools/issues)
- **Share this notebook** -- know someone who works with SEC data? Send them the Colab link

*edgartools is free, open-source, and community-driven. No API key or paid subscription required.*